In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ValueError: ignored

In [0]:
image = plt.imread('Bill_Gates_0000.jpg');
image = Image.fromarray(image)  
# Fit a la imagen
img_dim = (160,160);
image = ImageOps.fit(image,(img_dim[0],img_dim[1]),Image.ANTIALIAS,centering=(0.5, 0.5));
image = np.array(image).reshape(img_dim[1],img_dim[0],3) 
X = [];
X.append(image);
X = np.array(X)
res= facenet.predict(X);
print(res)

In [0]:
facenet_ext = Model(inputs=facenet.input, outputs=facenet.get_layer('Mixed_7a').output)
facenet_ext.summary()

In [0]:
embeding_train = facenet_ext.predict(ds.data_train['X']); 
embeding_test  = facenet_ext.predict(ds.data_test['X']); 

print('Embedings Train',embeding_train.shape)
print('Embedings Test',embeding_test.shape)

In [0]:
input_cla = Input(shape=(3, 3, 1792))
x=Reshape((3*3* 1792,), input_shape=(3, 3, 1792))(input_cla)
x=Dense(100,activation='relu', input_shape=(3*3*1792,))(x)
x=Dense(200,activation='relu')(x)
x=Dense(200,activation='relu')(x)
x=Dense(200,activation='relu')(x)
# output=Dense(5, activation='sigmoid')(x)
output=Dense(5, )(x)
adam = optimizers.adam(lr=1e-3)
classifier = Model(input_cla, output)
classifier.compile(optimizer=adam, loss=losses.mean_squared_error)
classifier.summary();

In [0]:
    h = classifier.fit(embeding_train, ds.data_train['Y'],
            epochs=1,
            batch_size=100,
            shuffle=False,
            validation_data=(embeding_test, ds.data_test['Y']))
    
    global_history['val_loss'].extend(h.history['val_loss'])
    global_history['loss'].extend(h.history['loss'])
    
    mean_acc = get_global_acc();
    global_history['mean_acc'].append(mean_acc);
    
    print('Mean-acc:',mean_acc)
    
    if mean_acc > best_acc:
        best_acc = mean_acc;
        save_model(index=save);

In [0]:
def test_factor_class(index=0):
    X = embeding_test;
    Y = ds.data_test['Y'];
    out = classifier.predict(X)
    
    limit = 0.5714; 
    # 4/7
    # limit = 0.5;
    
    acc = 0;

    for i in range(out.shape[0]): 
        if Y[i][index]>limit and out[i][index]>limit:
            acc = acc + 1;
        elif Y[i][index]<=limit and out[i][index]<=limit:
            acc = acc + 1;

    acc = acc / out.shape[0];
    acc *= 100;
    
    return acc

def get_global_acc(show=False):

    acc_global = 0;

    acc = test_factor_class(index=0);
    acc_global += acc;
    if show:
        print('O',acc);   

    acc = test_factor_class(index=1);
    acc_global += acc;
    if show:
        print('C',acc);    

    acc = test_factor_class(index=2);
    acc_global += acc;
    if show:
        print('E',acc);  

    acc = test_factor_class(index=3);
    acc_global += acc;
    if show:
        print('A',acc);    

    acc = test_factor_class(index=4);
    acc_global += acc;
    if show:
        print('N',acc);  

    acc_global = acc_global / 5;
    if show:
        print('\nPromedio',acc_global)
        
    return acc_global;

prom = get_global_acc(show=True);